## IMPORT MODULES

In [ ]:
from selection_service.enums.Enums import DesignCode, ProviderName
from selection_service.core.Pipeline import EarthquakeAPI
from selection_service.providers.ProvidersFactory import ProviderFactory
from selection_service.processing.Selection import SelectionConfig,SearchCriteria,TBDYSelectionStrategy,TargetParameters
from selection_service.core.LoggingConfig import setup_logging
import logging

## PREPROCESSING

In [ ]:
setup_logging(log_level= logging.INFO, log_dir= "logs")
con = SelectionConfig(design_code=DesignCode.TBDY_2018, num_records=22, max_per_station=3, max_per_event=3, min_score=55)
strategy = TBDYSelectionStrategy(config=con)
search_criteria = SearchCriteria(start_date="2020-01-01", end_date="2025-09-05", min_magnitude=7.0, max_magnitude=10.0, min_vs30=400, max_vs30=500 )
target_params = TargetParameters(magnitude=7.0, distance=30.0, vs30=400.0, pga=300, mechanism=["StrikeSlip","Reverse"] )

## API FUNC

In [ ]:
api = EarthquakeAPI(providerNames=[ProviderName.AFAD,
                                   ProviderName.PEER],
                    strategies=[strategy])

In [ ]:
result = api.run_sync(criteria=search_criteria,
                      target=target_params,
                      strategy_name=strategy.get_name())

In [ ]:
resultAsync = await api.run_async(criteria=search_criteria,
                            target=target_params,
                            strategy_name=strategy.get_name())
resultAsync

In [ ]:
resultAsync.value.scored_df

In [ ]:
type(result)

In [ ]:
type(result.value)

In [ ]:
result.success

In [ ]:
result.value.failed_providers

In [ ]:
result.value.execution_time

In [ ]:
result.value.logs

In [ ]:
target_params

In [ ]:
result.value.selected_df[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','MAGNITUDE_TYPE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

In [ ]:
result.value.scored_df[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

In [ ]:
result.value.scored_df.sort_values(by=['SCORE'],ascending=True)[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

In [ ]:
result.value.scored_df.describe().T

## FDSN Provider Test

In [ ]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta

BASE_URL = "https://service.iris.edu/fdsnws/event/1/query"

# Tarih aralığı: son 30 gün
endtime = datetime.utcnow()
starttime = endtime - timedelta(days=30)

params = {
    "starttime": starttime.strftime("%Y-%m-%dT%H:%M:%S"),
    "endtime": endtime.strftime("%Y-%m-%dT%H:%M:%S"),
    "minmagnitude": 4.0,
    "minlatitude": 35,
    "maxlatitude": 42,
    "minlongitude": 25,
    "maxlongitude": 45,
    "orderby": "time"
    # "format": "quakeml"  # <-- bunu kaldırdık
}

response = requests.get(BASE_URL, params=params)
response.raise_for_status()

# XML parse
root = ET.fromstring(response.text)

# QuakeML namespace
ns = {"q": "http://quakeml.org/xmlns/bed/1.2"}

events = []
for event in root.findall(".//q:event", ns):
    origin = event.find(".//q:origin", ns)
    magnitude = event.find(".//q:magnitude", ns)
    
    if origin is not None and magnitude is not None:
        time_el = origin.find(".//q:time/q:value", ns)
        lat_el = origin.find(".//q:latitude/q:value", ns)
        lon_el = origin.find(".//q:longitude/q:value", ns)
        depth_el = origin.find(".//q:depth/q:value", ns)
        mag_el = magnitude.find(".//q:mag/q:value", ns)

        events.append({
            "time": time_el.text if time_el is not None else None,
            "latitude": float(lat_el.text) if lat_el is not None else None,
            "longitude": float(lon_el.text) if lon_el is not None else None,
            "depth_km": float(depth_el.text)/1000 if depth_el is not None else None,
            "magnitude": float(mag_el.text) if mag_el is not None else None
        })

for ev in events:
    print(f"{ev['time']} | Mw {ev['magnitude']} | "
          f"Lat {ev['latitude']}, Lon {ev['longitude']}, Depth {ev['depth_km']} km")


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

BASE_URL = "https://service.iris.edu/fdsnws/event/1/query"

# Tarih aralığı: son 30 gün
endtime = datetime.utcnow()
starttime = endtime - timedelta(days=30)

params = {
    "starttime": starttime.strftime("%Y-%m-%dT%H:%M:%S"),
    "endtime": endtime.strftime("%Y-%m-%dT%H:%M:%S"),
    "minmagnitude": 4.0,
    "minlatitude": 35,
    "maxlatitude": 42,
    "minlongitude": 25,
    "maxlongitude": 45,
    "orderby": "time",
    "format": "geojson"
}

response = requests.get(BASE_URL, params=params)
response.raise_for_status()
data = response.json()

# Deprem bilgilerini çekelim
events = []
for feature in data.get("features", []):
    props = feature.get("properties", {})
    coords = feature.get("geometry", {}).get("coordinates", [None, None, None])
    
    events.append({
        "time": datetime.utcfromtimestamp(props.get("time", 0) / 1000),
        "magnitude": props.get("mag"),
        "place": props.get("place"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth_km": coords[2]
    })

# DataFrame'e dökelim
df = pd.DataFrame(events)
print(df)

# CSV'ye kaydetmek istersen:
# df.to_csv("earthquakes_turkey_last30days.csv", index=False)


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

BASE_URL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

# Son 30 gün
endtime = datetime.utcnow()
starttime = endtime - timedelta(days=30)

params = {
    "format": "geojson",
    "starttime": starttime.strftime("%Y-%m-%d"),
    "endtime": endtime.strftime("%Y-%m-%d"),
    "minmagnitude": 4.0,
    "minlatitude": 35,
    "maxlatitude": 42,
    "minlongitude": 25,
    "maxlongitude": 45,
    "orderby": "time"
}

response = requests.get(BASE_URL, params=params)
response.raise_for_status()
data = response.json()

events = []
for feature in data.get("features", []):
    props = feature.get("properties", {})
    coords = feature.get("geometry", {}).get("coordinates", [None, None, None])
    
    events.append({
        "time": datetime.utcfromtimestamp(props.get("time", 0) / 1000),
        "magnitude": props.get("mag"),
        "place": props.get("place"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth_km": coords[2]
    })

df = pd.DataFrame(events)
print(df.head())


In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime


class EarthquakeClient:
    PROVIDERS = {
        "IRIS": {
            "url": "https://service.iris.edu/fdsnws/event/1/query",
            "format": "xml"
        },
        "USGS": {
            "url": "https://earthquake.usgs.gov/fdsnws/event/1/query",
            "format": "geojson"
        }
    }

    def __init__(self, provider: str = "IRIS"):
        if provider not in self.PROVIDERS:
            raise ValueError(f"Desteklenmeyen provider: {provider}")
        self.provider = provider
        self.base_url = self.PROVIDERS[provider]["url"]
        self.format = self.PROVIDERS[provider]["format"]

    def search(self, **kwargs) -> pd.DataFrame:
        """
        Deprem araması yapar, DataFrame döner.
        Parametreler provider’a göre otomatik işlenir.
        """
        if self.provider == "USGS":
            return self._search_usgs(**kwargs)
        elif self.provider == "IRIS":
            return self._search_iris(**kwargs)

    def _search_usgs(self, **kwargs) -> pd.DataFrame:
        params = {"format": "geojson"}
        params.update(kwargs)

        r = requests.get(self.base_url, params=params)
        r.raise_for_status()
        data = r.json()

        events = []
        for feature in data.get("features", []):
            props = feature.get("properties", {})
            coords = feature.get("geometry", {}).get("coordinates", [None, None, None])
            events.append({
                "time": datetime.utcfromtimestamp(props.get("time", 0) / 1000),
                "magnitude": props.get("mag"),
                "place": props.get("place"),
                "longitude": coords[0],
                "latitude": coords[1],
                "depth_km": coords[2]
            })

        return pd.DataFrame(events)

    def _search_iris(self, **kwargs) -> pd.DataFrame:
        params = kwargs.copy()  # IRIS zaten XML (QuakeML) döner
        r = requests.get(self.base_url, params=params)
        r.raise_for_status()
        root = ET.fromstring(r.text)

        ns = {"q": "http://quakeml.org/xmlns/bed/1.2"}
        events = []
        for event in root.findall(".//q:event", ns):
            origin = event.find(".//q:origin", ns)
            magnitude = event.find(".//q:magnitude", ns)
            if origin is None or magnitude is None:
                continue
            time_el = origin.find(".//q:time/q:value", ns)
            lat_el = origin.find(".//q:latitude/q:value", ns)
            lon_el = origin.find(".//q:longitude/q:value", ns)
            depth_el = origin.find(".//q:depth/q:value", ns)
            mag_el = magnitude.find(".//q:mag/q:value", ns)

            events.append({
                "time": time_el.text if time_el is not None else None,
                "latitude": float(lat_el.text) if lat_el is not None else None,
                "longitude": float(lon_el.text) if lon_el is not None else None,
                "depth_km": float(depth_el.text) / 1000 if depth_el is not None else None,
                "magnitude": float(mag_el.text) if mag_el is not None else None
            })

        return pd.DataFrame(events)

# USGS üzerinden GeoJSON (Türkiye sınırları, son 7 gün, Mw >= 4.5)
client = EarthquakeClient(provider="USGS")
df = client.search(
    starttime="2025-09-18",
    endtime="2025-09-25",
    minmagnitude=4.5,
    minlatitude=35,
    maxlatitude=42,
    minlongitude=25,
    maxlongitude=45,
    orderby="time"
)
print(df.head())


# IRIS üzerinden QuakeML (aynı parametreler)
client = EarthquakeClient(provider="IRIS")
df2 = client.search(
    starttime="2025-09-18T00:00:00",
    endtime="2025-09-25T23:59:59",
    minmagnitude=4.5,
    minlatitude=35,
    maxlatitude=42,
    minlongitude=25,
    maxlongitude=45,
    orderby="time"
)
print(df2.head())


In [ ]:
from obspy.clients.fdsn import Client
fdsn = Client("IRIS")
catalog = fdsn.get_events(starttime="2020-01-01", endtime="2025-09-05", minmagnitude=7.0, maxmagnitude=10.0)
catalog

In [ ]:
catalog.events[0].preferred_origin()

In [ ]:
catalog.events[0].preferred_magnitude()

In [ ]:
catalog.events[0].preferred_focal_mechanism()

In [ ]:
from obspy import UTCDateTime


t1 = UTCDateTime("2010-02-27T06:30:00.000")
t2 = t1 + 5
st = fdsn.get_waveforms("IU", "ANMO", "00", "LHZ", t1, t2)


In [ ]:
st.traces[0].stats

## DOWNLOAD AFAD WAVEFORMS

In [ ]:
result.value.selected_df.columns

In [ ]:
afad_filenames = result.value.selected_df[result.value.selected_df['PROVIDER'] == "AFAD"]['FILE_NAME_H1'].to_list()
afad_filenames

In [ ]:
afad_provider = api.providers[0]
response= afad_provider.download_afad_waveforms_batch(afad_filenames,file_status="RawAcc", batch_size=5, event_id="TEST_EVENT", user_name="GuestUser", export_type= "mseed")
response

In [ ]:
response.value

In [ ]:
# file_path="D:\\github\\SelectionEarthquake\\examples\\Afad_events\\TEST_EVENT\\0213\\20230206011732_0213.mseed"
file_path="D:\\github\\SelectionEarthquake\\examples\\Afad_events\\TEST_EVENT\\0213\\20230206011732_0213.mseed"

In [ ]:
# mseed_reader.py
import numpy as np
import re
from typing import Dict, List

class GroundMotionReader():
    """
    AFAD'ın metin tabanlı deprem veri formatını okuyan sınıf.
    """
    def __init__(self, file_path):
        self.file_path = file_path
        self.metadata = {}
        self.data = None
        self.units = "gal"
        self.fs = 100.0  # Varsayılan örnekleme frekansı (0.01 s → 100 Hz)

    def start(self):
        """Veri dosyasını okur ve verileri hazırlar."""
        self._parse_file()

    def _parse_file(self):
        """Dosyayı satır satır okuyarak metadata ve verileri ayırır."""
        with open(self.file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        metadata_lines = []
        data_lines = []
        in_data_section = False
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            if line.startswith(('N-S', 'E-W', 'U-D')) and not in_data_section:
                # Veri bölümü başlığı
                in_data_section = True
                continue
                
            if in_data_section:
                # Veri satırları
                if re.match(r'^-?\d+\.\d+', line):
                    data_lines.append(line)
            else:
                # Metadata satırları
                metadata_lines.append(line)
        
        # Metadata'ları parse et
        self._parse_metadata(metadata_lines)
        
        # Verileri parse et
        self._parse_data(data_lines)

    def _parse_metadata(self, lines: List[str]):
        """Metadata satırlarını parse eder."""
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()
                self.metadata[key] = value
                
                # Önemli bilgileri ayıkla
                if 'SAMPLING INTERVAL' in key:
                    try:
                        interval = float(value.split()[0])
                        self.fs = 1.0 / interval  # Örnekleme frekansı
                    except:
                        pass
                elif 'NUMBER OF DATA' in key:
                    try:
                        self.n_data = int(value)
                    except:
                        pass
                elif 'RAW PGA VALUES' in key:
                    # PGA değerlerini ayıkla
                    pga_values = re.findall(r'\(([NSEWUD\-]+)\)\s+([\d\.]+)', value)
                    for direction, value in pga_values:
                        self.metadata[f'PGA_{direction}'] = float(value)

    def _parse_data(self, lines: List[str]):
        """Veri satırlarını parse eder."""
        ns_data = []
        ew_data = []
        ud_data = []
        
        for line in lines:
            # Satırdaki 3 float değerini ayıkla
            values = re.findall(r'-?\d+\.\d+', line)
            if len(values) == 3:
                ns_data.append(float(values[0]))
                ew_data.append(float(values[1]))
                ud_data.append(float(values[2]))
        
        # NumPy array'lerine dönüştür
        self.data = {
            'NS': np.array(ns_data),
            'EW': np.array(ew_data),
            'UD': np.array(ud_data)
        }
        
        # Zaman vektörünü oluştur
        n_samples = len(ns_data)
        self.time = np.arange(0, n_samples / self.fs, 1 / self.fs)

    def get_data(self) -> Dict:
        """Parselenen verileri döndürür."""
        if self.data is None:
            raise ValueError("Önce start() metodunu çağırın")
        
        return {
            "direction": ['NS', 'EW', 'UD'],
            "time": self.time,
            "acc": np.array([self.data['NS'], self.data['EW'], self.data['UD']]),
            "units": self.units,
            "fs": self.fs,
            "meta_data": self.metadata,
            "n_samples": len(self.time)
        }

    def stop(self):
        """Temizlik işlemleri."""
        pass

# Test

try:
    reader = GroundMotionReader(file_path)
    reader.start()
    data = reader.get_data()
    
    print("✓ Veri başarıyla okundu!")
    print(f"Örnek sayısı: {data['n_samples']}")
    print(f"Örnekleme frekansı: {data['fs']} Hz")
    print(f"Birimler: {data['units']}")
    print(f"Zaman aralığı: {data['time'][0]:.2f} - {data['time'][-1]:.2f} s")
    print(f"NS veri boyutu: {data['acc'][0].shape}")
    print(f"EW veri boyutu: {data['acc'][1].shape}")
    print(f"UD veri boyutu: {data['acc'][2].shape}")
    
    # İlk 5 örnek
    print("\nİlk 5 örnek:")
    for i in range(5):
        print(f"{data['time'][i]:6.3f}s: NS={data['acc'][0][i]:8.3f}, EW={data['acc'][1][i]:8.3f}, UD={data['acc'][2][i]:8.3f} {data['units']}")
    
    # Metadata
    print("\nÖnemli Metadata:")
    for key in ['PLACE', 'EARTHQUAKE DATE', 'EARTHQUAKE MAGNITUDE', 'STATION ID']:
        if key in data['meta_data']:
            print(f"{key}: {data['meta_data'][key]}")
            
except Exception as e:
    print(f"✗ Hata: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(data['acc'][2])

In [ ]:

from selection_service.processing.Selection import SearchCriteria,TargetParameters
from selection_service.enums.Enums import ProviderName

prvFactory = ProviderFactory()
fdsnProvider = prvFactory.create_provider(ProviderName.FDSN)
search_crit = SearchCriteria(start_date="2020-01-01", end_date="2025-09-05", min_magnitude=7.0, max_magnitude=10.0, min_vs30=400, max_vs30=500 )
result = await fdsnProvider.fetch_data_async(criteria=search_crit)
result

In [ ]:
result.value.iloc()[0]['EQID']

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

# IRIS istemcisini oluştur
client = Client("IRIS")

# Deprem parametrelerini belirle
starttime = UTCDateTime("2024-01-01T00:00:00")
endtime = UTCDateTime("2025-01-02T00:00:00")
min_latitude = 35.0
max_latitude = 42.0
min_longitude = 25.0
max_longitude = 45.0
min_magnitude = 4.0

# Deprem kataloğunu çek
catalog = client.get_events(
    starttime=starttime,
    endtime=endtime,
    minlatitude=min_latitude,
    maxlatitude=max_latitude,
    minlongitude=min_longitude,
    maxlongitude=max_longitude,
    minmagnitude=min_magnitude
)

print(f"Bulunan deprem sayısı: {len(catalog)}")

# Depremleri listele
for i, event in enumerate(catalog):
    origin = event.preferred_origin()
    magnitude = event.preferred_magnitude()
    print(f"{i+1}. Deprem:")
    print(f"   Zaman: {origin.time}")
    print(f"   Enlem: {origin.latitude:.2f}")
    print(f"   Boylam: {origin.longitude:.2f}")
    print(f"   Derinlik: {origin.depth/1000:.1f} km")
    print(f"   Büyüklük: {magnitude.mag}")
    print("-" * 50)

In [ ]:
dir(catalog[0])

In [ ]:
type(catalog[0])

In [ ]:
catalog[0].__dict__

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt

def deprem_dalga_formu_cek(deprem_zamani, istasyon, sure=300):
    """Belirli bir depremin dalga formunu çek"""
    client = Client("IRIS")
    
    # Depremden sonraki dalga formlarını çek
    start = deprem_zamani - 30  # Depremden 30 saniye önce
    end = start + sure  # Toplam süre
    
    try:
        # Dalga formu verisini çek
        stream = client.get_waveforms(
            network="IU",  # Global istasyon ağı
            station=istasyon,
            location="00",
            channel="BHZ",  # Düşey bileşen
            starttime=start,
            endtime=end
        )
        
        # Veriyi işle
        stream.detrend('linear')
        stream.taper(max_percentage=0.05)
        stream.filter('bandpass', freqmin=0.01, freqmax=1.0)
        
        # Grafik çiz
        plt.figure(figsize=(12, 6))
        for i, trace in enumerate(stream):
            plt.plot(trace.times(), trace.data, label=trace.id)
        
        plt.axvline(x=30, color='red', linestyle='--', label='Deprem Zamanı')
        plt.xlabel('Zaman (s)')
        plt.ylabel('Genlik')
        plt.title(f'Deprem Dalga Formu - İstasyon: {istasyon}')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        return stream
        
    except Exception as e:
        print(f"Hata: {e}")
        return None

# Örnek kullanım
deprem_zamani = UTCDateTime("2024-01-01T10:30:00")
deprem_dalga_formu_cek(deprem_zamani,None, 300)